## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

https://www.informatik.uni-leipzig.de/~saeedi/NCVoters_Readme.txt

```
5 party:
--------- 
5 sources. 
Each source 1000,000 entities.
There is one file per source, so totally 5 files
****************************************************
****************************************************
****************************************************
Fields:
---------- 
recId: entites with the same recId refer to the same entity.
givenname: 
surname: 
post code: 
suburb:
```

In [6]:
import glob
import csv
from tqdm.auto import tqdm

current_row_id = 0
row_dict = {}
rows_total = 5000000
cluster_attr = 'recid'

with tqdm(total=rows_total) as pbar:
    for filename in glob.glob(f'{home_dir}/Downloads/5Party-ocp20/*.csv'):
        with open(filename) as f:
            for row in csv.DictReader(f):
                row['id'] = current_row_id
                row[cluster_attr] = int(row[cluster_attr])  # convert cluster_attr to int
                row_dict[current_row_id] = row
                current_row_id += 1
                pbar.update(1)

In [7]:
row_dict[0]

{'recid': 7852009,
 'givenname': 'kadelyn',
 'surname': 'gragnani',
 'suburb': 'waxhaw',
 'postcode': '28|73',
 'id': 0}

## Preprocess

In [8]:
attr_list = ['givenname', 'surname', 'suburb', 'postcode']

In [9]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()[:30]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'givenname': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'surname': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'suburb': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'postcode': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_encoder

row_encoder = build_row_encoder(attr_info_dict, row_dict=row_dict)
row_encoder.attr_info_dict

20:49:31 INFO:For attr='givenname', computing actual alphabet and max_str_len
20:49:34 INFO:For attr='givenname', using actual_max_str_len=16
20:49:34 INFO:For attr='surname', computing actual alphabet and max_str_len
20:49:37 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
20:49:37 INFO:For attr='surname', using actual_max_str_len=22
20:49:37 INFO:For attr='suburb', computing actual alphabet and max_str_len
20:49:40 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
20:49:40 INFO:For attr='suburb', using actual_max_str_len=22
20:49:40 INFO:For attr='postcode', computing actual alphabet and max_str_len
20:49:43 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
20:49:43 INFO:For attr='postcode', using actual_max_str_len=10


{'givenname': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16),
 'surname': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22),
 'suburb': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7

In [14]:
from entity_embed import DeduplicationDataModule

train_cluster_len = 1000
valid_cluster_len = 1000
test_cluster_len = 200_000
datamodule = DeduplicationDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_encoder=row_encoder,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=test_cluster_len,
    only_plural_clusters=True,
    log_empty_vals=False,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import EntityEmbed

ann_k = 10
model = EntityEmbed(
    datamodule,
    ann_k=ann_k,
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.5',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'voters'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

20:49:43 INFO:GPU available: True, used: True
20:49:43 INFO:TPU available: None, using: 0 TPU cores
20:49:43 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

20:49:55 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
20:49:57 INFO:Train pair count: 6679
20:49:57 INFO:Valid pair count: 6645
20:49:57 INFO:Test pair count: 1331364
20:50:00 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 2.5 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params


1

In [18]:
model.blocker_net.get_signature_weights()

{'givenname': 0.23608161509037018,
 'surname': 0.31067323684692383,
 'suburb': 0.2341073453426361,
 'postcode': 0.21913784742355347}

In [19]:
import gc
del trainer
gc.collect()

57

## Testing manually 

In [20]:
datamodule.setup(stage='test')

20:51:17 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
20:51:19 INFO:Train pair count: 6679
20:51:19 INFO:Valid pair count: 6645
20:51:19 INFO:Test pair count: 1331364


In [21]:
test_row_dict = datamodule.test_row_dict
test_vector_dict = model.predict(
    row_dict=test_row_dict,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [23]:
import gc
del model
del datamodule
gc.collect()

2618

In [24]:
assert len(test_vector_dict) == len(test_row_dict)

In [25]:
%%time

from entity_embed import ANNEntityIndex

ann_index = ANNEntityIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(test_vector_dict)
ann_index.build()

CPU times: user 15min 58s, sys: 2.39 s, total: 16min
Wall time: 1min 31s


In [26]:
%%time

sim_threshold = 0.5
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold
)

CPU times: user 21min 40s, sys: 746 ms, total: 21min 41s
Wall time: 1min 56s


In [27]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

6.158318047481651

In [28]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.27037957956151937, 0.9995448277105284)

In [29]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

3591056

In [30]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

606

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_vector_dict[id_left], test_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.95234567,
 {'recid': 3463074,
  'givenname': 'eric',
  'surname': 'smith',
  'suburb': 'carolina beach',
  'postcode': '28428',
  'id': 1861067},
 {'recid': 3463074,
  'givenname': 'erc',
  'surname': 'smith',
  'suburb': 'carolina beach',
  'postcode': 'z8428',
  'id': 2060752})

(0.6916543,
 {'recid': 3457209,
  'givenname': 'aggie',
  'surname': 'fillips',
  'suburb': 'wadesb0ro',
  'postcode': '28170',
  'id': 2045963},
 {'recid': 3457209,
  'givenname': 'aggie',
  'surname': 'phillips',
  'suburb': 'wadesboro',
  'postcode': '28170',
  'id': 4697033})

(0.442439,
 {'recid': 3259167,
  'givenname': 'lindsay',
  'surname': 'xmeltzer',
  'suburb': 'salisbury',
  'postcode': '2814q',
  'id': 4155},
 {'recid': 3259167,
  'givenname': 'lindsay',
  'surname': 'schmeltzer',
  'suburb': 'salisbury',
  'postcode': '28144',
  'id': 1742878})

(0.9999998,
 {'recid': 6651354,
  'givenname': 'phillip',
  'surname': 'evans',
  'suburb': 'carolina beach',
  'postcode': '28428',
  'id': 654557},
 {'recid': 6651354,
  'givenname': 'phillip',
  'surname': 'evans',
  'suburb': 'carolina beach',
  'postcode': '28428',
  'id': 2654370})

(0.6108625,
 {'recid': 932111,
  'givenname': 'robert',
  'surname': 'fillips',
  'suburb': 'duhn',
  'postcode': '28334',
  'id': 3063662},
 {'recid': 932111,
  'givenname': 'robert',
  'surname': 'phillips',
  'suburb': 'dunn',
  'postcode': '28334',
  'id': 4131364})

(0.830707,
 {'recid': 3077273,
  'givenname': 'timothy',
  'surname': 'alexandeah',
  'suburb': 'charlotte',
  'postcode': '2826z',
  'id': 1034118},
 {'recid': 3077273,
  'givenname': 'timothy',
  'surname': 'alexander',
  'suburb': 'charlotte',
  'postcode': '28262',
  'id': 4552594})

(0.62918615,
 {'recid': 8075424,
  'givenname': "cather'lne",
  'surname': 'matthews',
  'suburb': 'charlotte',
  'postcode': '28249',
  'id': 2036282},
 {'recid': 8075424,
  'givenname': 'caterine',
  'surname': 'matthews',
  'suburb': 'charlote',
  'postcode': '28209',
  'id': 4036524})

(0.63899416,
 {'recid': 5584048,
  'givenname': 'angel',
  'surname': 'jones',
  'suburb': 'roanocke rapids',
  'postcode': '2787o',
  'id': 1042238},
 {'recid': 5584048,
  'givenname': 'ankel',
  'surname': 'jories',
  'suburb': 'roanoke rapids',
  'postcode': '27870',
  'id': 4042346})

(0.94088894,
 {'recid': 5797641,
  'givenname': 'crystal',
  'surname': 'durham',
  'suburb': 'henderson',
  'postcode': '27536',
  'id': 2366027},
 {'recid': 5797641,
  'givenname': 'crystal',
  'surname': 'durhsm',
  'suburb': 'henderso',
  'postcode': '27536',
  'id': 4028591})

(0.91307825,
 {'recid': 1395938,
  'givenname': 'ane',
  'surname': 'alexander',
  'suburb': 'elizabet city',
  'postcode': '27909',
  'id': 2001114},
 {'recid': 1395938,
  'givenname': 'anne',
  'surname': 'alexander',
  'suburb': 'elizabeth city',
  'postcode': '27909',
  'id': 3940098})